## Summing all the prime numbers below a given number

In [5]:
import time
import sys

# Simple code

def if_prime(x):
    if x <= 1:
        return 0
    elif x <= 3:
        return x
    elif x % 2 == 0 or x % 3 == 0:
        return 0
    i = 5
    while i**2 <= x:
        if x % i == 0 or x % (i + 2) == 0:
            return 0
        i += 6
    return x

def sum_primes(x):
    result = 0
    for i in range(x):
        result += if_prime(i)
    return result

# Valor por defecto
number = 2500000

# Si se pasa un argumento, lo usamos
if len(sys.argv) > 1:
    try:
        number = int(sys.argv[1])
        print(f"Recibido argumento number: {number}")
    except ValueError:
        pass
suma = 0
N = 3 # number of loops

start = time.time()
for i in range(N):
    suma = sum(map(if_prime, list(range(number))))
stop = time.time()
tiempo = (stop - start) / N

print("The prime sum below ", number, "is ", suma, " and the time taken is", tiempo)

tiempo = %timeit -r 2 -o -q sum_primes(number)
suma = sum_primes(number)
print("The prime sum below ", number, "is ", suma, " and the time taken is", tiempo)


The prime sum below  2500000 is  219697708195  and the time taken is 4.8556482791900635
The prime sum below  2500000 is  219697708195  and the time taken is 4.92 s ± 44.9 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)


In [6]:
# --- CELDA DE CÓDIGO ---

import time
import sys
import os
import multiprocessing
import numpy as np
from numba import njit, prange

# ==========================================
# 1. GESTIÓN DE ARGUMENTOS (Para 3.2.d)
# ==========================================
# Valor por defecto (para cuando ejecutas en Jupyter normal)
number = 2_500_000 

# Intentamos leer el argumento desde la línea de comandos (para cuando lo lanza SLURM)
if len(sys.argv) > 1:
    try:
        # sys.argv[1] es el primer argumento pasado al script
        param = int(sys.argv[1])
        if param > 0:
            number = param
            print(f"--> [ARGUMENTO] Ejecutando con number = {number}")
    except (ValueError, IndexError):
        print(f"--> [INFO] No se detectó argumento válido. Usando defecto: {number}")
else:
    print(f"--> [INFO] Sin argumentos externos. Usando defecto: {number}")

# ==========================================
# 2. DEFINICIÓN DE FUNCIONES
# ==========================================

# Función base optimizada con Numba (usada por Secuencial y Numba Paralelo)
@njit(fastmath=True)
def if_prime_numba(x):
    if x <= 1: return False
    if x <= 3: return True
    if x % 2 == 0 or x % 3 == 0: return False
    i = 5
    while i**2 <= x:
        if x % i == 0 or x % (i + 2) == 0:
            return False
        i += 6
    return True

# Función wrapper necesaria para Multiprocessing (debe estar en el nivel superior)
def worker_prime(x):
    if if_prime_numba(x):
        return x
    return 0

# Función específica para Numba Paralelo (prange)
@njit(parallel=True)
def sum_primes_numba_par(n):
    total = 0
    # prange paraleliza automáticamente este bucle
    for i in prange(n):
        if if_prime_numba(i):
            total += i
    return total

# ==========================================
# 3. EJECUCIÓN Y MEDICIÓN
# ==========================================

if __name__ == '__main__':
    print(f"\n=== Iniciando pruebas para N = {number} ===\n")

    # --- A) NUMBA SECUENCIAL ---
    print("--- A) Numba Secuencial ---")
    start = time.time()
    # Ejecutamos un bucle simple llamando a la función compilada
    suma_seq = 0
    for i in range(number):
        if if_prime_numba(i):
            suma_seq += i
    stop = time.time()
    print(f"Resultado: {suma_seq}")
    print(f"Tiempo: {stop - start:.4f} s\n")

    # --- B) MULTIPROCESSING CON POOL ---
    print("--- B) Multiprocessing ---")
    
    # Leemos la variable de entorno que manda el script SLURM
    # Si no existe (ejecución local), usa 4 por defecto
    n_cores_env = int(os.environ.get('OMP_NUM_THREADS', 4))
    print(f"Usando {n_cores_env} procesos (cores)...")
    
    start = time.time()
    # Creamos el pool con el número exacto de cores que pide el script
    with multiprocessing.Pool(processes=n_cores_env) as pool:
        # Usamos chunksize para mejorar rendimiento
        results = pool.map(worker_prime, range(number), chunksize=2000)
        suma_mp = sum(results)
    
    stop = time.time()
    print(f"Resultado: {suma_mp}")
    print(f"Tiempo: {stop - start:.4f} s\n")

    # --- C) NUMBA PARALELO (prange) ---
    print("--- C) Numba Paralelo (prange) ---")
    # Warm-up (compilación)
    _ = sum_primes_numba_par(100)
    
    start = time.time()
    # Numba lee automáticamente OMP_NUM_THREADS, no hace falta pasárselo
    suma_nb = sum_primes_numba_par(number)
    stop = time.time()
    print(f"Resultado: {suma_nb}")
    print(f"Tiempo: {stop - start:.4f} s\n")

--> [INFO] No se detectó argumento válido. Usando defecto: 2500000

=== Iniciando pruebas para N = 2500000 ===

--- A) Numba Secuencial ---
Resultado: 219697708195
Tiempo: 0.6851 s

--- B) Multiprocessing ---
Usando 4 procesos (cores)...
Resultado: 219697708195
Tiempo: 0.2198 s

--- C) Numba Paralelo (prange) ---
Resultado: 219697708195
Tiempo: 0.0362 s



# 3.2. e)

Iniciando trabajo SLURM: 12939

Convirtiendo notebook a script python...
/nas/hdd-0/modules/anaconda3-2025/bin/jupyter-nbconvert:7: DeprecationWarning: Parsing dates involving a day of month without a year specified is ambiguious
and fails to parse leap day. The default behavior will change in Python 3.15
to either always raise an exception or to use a different default year (TBD).
To avoid trouble, add a specific year to the input & format.
See https://github.com/python/cpython/issues/70647.
  from nbconvert.nbconvertapp import main
[NbConvertApp] Converting notebook primes-par-alumno06.ipynb to python
[NbConvertApp] Writing 4704 bytes to primes-par-alumno06.py
 
#############################################
    EJECUTANDO PARA TAMAÑO N = 1000000
#############################################
 
>>> Configurando entorno para 1 núcleos <<<
Recibido argumento number: 1000000
The prime sum below  1000000 is  37550402023  and the time taken is 2.2453622023264566
The prime sum below  1000000 is  37550402023  and the time taken is 2.19 s ± 62.7 μs per loop (mean ± std. dev. of 2 runs, 1 loop each)
--> [ARGUMENTO] Ejecutando con number = 1000000

=== Iniciando pruebas para N = 1000000 ===

--- A) Numba Secuencial ---
Resultado: 37550402023
Tiempo: 4.7847 s

--- B) Multiprocessing ---
Usando 1 procesos (cores)...
Resultado: 37550402023
Tiempo: 0.4922 s

--- C) Numba Paralelo (prange) ---
Resultado: 37550402023
Tiempo: 0.0082 s

 
>>> Configurando entorno para 2 núcleos <<<
Recibido argumento number: 1000000
The prime sum below  1000000 is  37550402023  and the time taken is 2.3067499796549478
The prime sum below  1000000 is  37550402023  and the time taken is 2.3 s ± 12.1 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
--> [ARGUMENTO] Ejecutando con number = 1000000

=== Iniciando pruebas para N = 1000000 ===

--- A) Numba Secuencial ---
Resultado: 37550402023
Tiempo: 1.3056 s

--- B) Multiprocessing ---
Usando 2 procesos (cores)...
Resultado: 37550402023
Tiempo: 0.2485 s

--- C) Numba Paralelo (prange) ---
Resultado: 37550402023
Tiempo: 0.0228 s

 
>>> Configurando entorno para 4 núcleos <<<
Recibido argumento number: 1000000
The prime sum below  1000000 is  37550402023  and the time taken is 2.2689327398935952
The prime sum below  1000000 is  37550402023  and the time taken is 2.2 s ± 289 μs per loop (mean ± std. dev. of 2 runs, 1 loop each)
--> [ARGUMENTO] Ejecutando con number = 1000000

=== Iniciando pruebas para N = 1000000 ===

--- A) Numba Secuencial ---
Resultado: 37550402023
Tiempo: 1.1456 s

--- B) Multiprocessing ---
Usando 4 procesos (cores)...
Resultado: 37550402023
Tiempo: 0.1424 s

--- C) Numba Paralelo (prange) ---
Resultado: 37550402023
Tiempo: 0.0343 s

 
>>> Configurando entorno para 8 núcleos <<<
Recibido argumento number: 1000000
The prime sum below  1000000 is  37550402023  and the time taken is 2.248943487803141
The prime sum below  1000000 is  37550402023  and the time taken is 2.18 s ± 38.7 μs per loop (mean ± std. dev. of 2 runs, 1 loop each)
--> [ARGUMENTO] Ejecutando con number = 1000000

=== Iniciando pruebas para N = 1000000 ===

--- A) Numba Secuencial ---
Resultado: 37550402023
Tiempo: 1.3786 s

--- B) Multiprocessing ---
Usando 8 procesos (cores)...
Resultado: 37550402023
Tiempo: 0.1340 s

--- C) Numba Paralelo (prange) ---
Resultado: 37550402023
Tiempo: 0.0156 s

 
#############################################
    EJECUTANDO PARA TAMAÑO N = 10000000
#############################################
 
>>> Configurando entorno para 1 núcleos <<<
Recibido argumento number: 10000000
The prime sum below  10000000 is  3203324994356  and the time taken is 59.01686684290568
The prime sum below  10000000 is  3203324994356  and the time taken is 57.5 s ± 12.8 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
--> [ARGUMENTO] Ejecutando con number = 10000000

=== Iniciando pruebas para N = 10000000 ===

--- A) Numba Secuencial ---
Resultado: 3203324994356
Tiempo: 5.8079 s

--- B) Multiprocessing ---
Usando 1 procesos (cores)...
Resultado: 3203324994356
Tiempo: 6.4533 s

--- C) Numba Paralelo (prange) ---
Resultado: 3203324994356
Tiempo: 0.1497 s

 
>>> Configurando entorno para 2 núcleos <<<
Recibido argumento number: 10000000
The prime sum below  10000000 is  3203324994356  and the time taken is 59.462855418523155
The prime sum below  10000000 is  3203324994356  and the time taken is 57.4 s ± 14.3 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
--> [ARGUMENTO] Ejecutando con number = 10000000

=== Iniciando pruebas para N = 10000000 ===

--- A) Numba Secuencial ---
Resultado: 3203324994356
Tiempo: 6.7602 s

--- B) Multiprocessing ---
Usando 2 procesos (cores)...
Resultado: 3203324994356
Tiempo: 2.9608 s

--- C) Numba Paralelo (prange) ---
Resultado: 3203324994356
Tiempo: 0.1759 s

 
>>> Configurando entorno para 4 núcleos <<<
Recibido argumento number: 10000000
The prime sum below  10000000 is  3203324994356  and the time taken is 59.8850367863973
The prime sum below  10000000 is  3203324994356  and the time taken is 57.5 s ± 26.3 μs per loop (mean ± std. dev. of 2 runs, 1 loop each)
--> [ARGUMENTO] Ejecutando con number = 10000000

=== Iniciando pruebas para N = 10000000 ===

--- A) Numba Secuencial ---
Resultado: 3203324994356
Tiempo: 6.6435 s

--- B) Multiprocessing ---
Usando 4 procesos (cores)...
Resultado: 3203324994356
Tiempo: 1.5645 s

--- C) Numba Paralelo (prange) ---
Resultado: 3203324994356
Tiempo: 0.1347 s

 
>>> Configurando entorno para 8 núcleos <<<
Recibido argumento number: 10000000
The prime sum below  10000000 is  3203324994356  and the time taken is 59.545202573140465
The prime sum below  10000000 is  3203324994356  and the time taken is 57.3 s ± 6.78 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
--> [ARGUMENTO] Ejecutando con number = 10000000

=== Iniciando pruebas para N = 10000000 ===

--- A) Numba Secuencial ---
Resultado: 3203324994356
Tiempo: 6.5190 s

--- B) Multiprocessing ---
Usando 8 procesos (cores)...
Resultado: 3203324994356
Tiempo: 1.0785 s

--- C) Numba Paralelo (prange) ---
Resultado: 3203324994356
Tiempo: 0.1555 s

Trabajo finalizado.


# 3.2. e) 

- Código original (python) : El tiempo escala linealmente con el tamaño del problema (al multiplicar N por 10, el tiempo pasa de 2.2s a 59s, aprox x27, lo cual es coherente con la complejidad de buscar primos). Comportamiento con Cores: No mejora absolutamente nada al aumentar los núcleos (siempre tarda ~2.2s o ~59s). Esto se debe al GIL (Global Interpreter Lock) de Python, que impide que un script básico utilice más de un hilo de CPU simultáneamente.

- Numba Secuencial : simplemente usando @njit, mejora de casi 10x solo por compilar el código a lenguaje máquina, eliminando la sobrecarga del intérprete de Python. Al igual que el código original, el tiempo no baja al añadir núcleos (se mantiene estable en ~6.5s). Esto es esperado, ya que la función no está paralelizada (parallel=False por defecto).

- Multiprocessing (Paralelismo de Procesos) : Es muy eficaz para paralelizar tareas pesadas. Observamos que pasar de 4 a 8 núcleos no reduce el tiempo a la mitad (de 1.5s baja a 1.0s, no a 0.75s). Esto se debe al overhead (sobrecarga) de gestionar 8 procesos y comunicar los resultados de vuelta al proceso principal. Para un problema de este tamaño, 8 núcleos empiezan a saturar la gestión de procesos.

- Numba Paralelo (prange + parallel=True) : Es increíblemente rápido. Numba con parallel=True aplica optimizaciones de vectorización (SIMD) y uso de caché muy agresivas (fastmath=True) que aceleran el código masivamente incluso en un solo hilo.


Para cálculo numérico intensivo en Python, la combinación de compilación JIT (Numba) con paralelismo de hilos (prange) es, por mucho, la opción más eficiente, superando al paralelismo basado en procesos (multiprocessing) debido a su menor sobrecarga de memoria y gestión.